TO DO:
- supervised ce? 
- replicate w/ twitter dfs? 
- try different types of scoring in logreg?  

In [1]:
#imports + path
import pandas as pd
import json
import re
import numpy as np
import category_encoders as ce
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.feature_extraction.text import TfidfVectorizer
from pandas_profiling import ProfileReport
path = '/Users/laurentfintoni/Desktop/University/COURSE DOCS/THESIS/Internship/musow-pipeline/LOGREG_RELEVANCE/'

Part 1: 
- create DFs for trainin set using musoW and MJI spreadsheets
- two versions: a base one and one w/ additions from ismir dataset (for musoW) and from additional research (for MJI)
- the baseline difference we want to train the LogReg model on is that musoW is focused on music archives w/ datasets while MJI is focused on all sorts of music archives, regardless of dataset inclusion
- to avoid polluting the two sets we check for duplicates at all possible stages to ensure that the MJI set does not have anything that currently exists in musoW 

In [2]:
#read mji csv and grab needed columns
df_mji = pd.read_csv('/Users/laurentfintoni/Desktop/University/COURSE DOCS/THESIS/Internship/musow-pipeline/MJI/MJI_data.csv', keep_default_na=False, dtype='string')
df_mji_small = pd.DataFrame(columns=['Title', 'Description', 'URL'])
df_mji_small['Title'] = df_mji['Title'].str.lower().str.strip()
df_mji_small['Description'] = df_mji['Description'].str.lower().str.strip()
df_mji_small['URL'] = df_mji['URL'].str.lower().str.strip()

In [3]:
#read musow json dump and grab needed columns
with open('/Users/laurentfintoni/Desktop/University/COURSE DOCS/THESIS/Internship/musow-pipeline/MUSOW/musow_name_desc_url_cat.json') as file:
    data = json.load(file)
    
musow_names = [result['name']['value'].strip().lower() for result in data['results']['bindings']]
musow_desc = [result['description']['value'].strip().lower() for result in data['results']['bindings']]
musow_url = [result['url']['value'].strip().lower() for result in data['results']['bindings']]
df_musow = pd.DataFrame(columns=['Title', 'Description', 'URL'])
df_musow['Title'] = musow_names
df_musow['Description'] = musow_desc
df_musow['URL'] = musow_url
df_musow = df_musow.astype('string')

In [4]:
#remove musow duplicates from MJI set 
mji_training_set = df_mji_small[~df_mji_small['Title'].isin(df_musow['Title'])].dropna()

In [5]:
#create positive and negative sets w/o additions, add target column 
positive_df = df_musow.copy()
positive_df['Target'] = '1'
negative_df = mji_training_set.copy()
negative_df['Target'] = '0'

In [6]:
#create positive and negative sets w/ additions, add target column 
ismir_df = pd.read_pickle('/Users/laurentfintoni/Desktop/University/COURSE DOCS/THESIS/Internship/musow-pipeline/GH_PICKLES/ismir.pkl')
ismir_df = ismir_df[~ismir_df['Title'].isin(df_musow['Title'])].dropna() 
positive_df_adds = pd.concat([df_musow, ismir_df]).reset_index(drop=True)
positive_df_adds = positive_df_adds.drop_duplicates(['Title'], keep='last')
positive_df_adds['Target'] = '1'
mji_additions_1 = pd.read_csv('/Users/laurentfintoni/Desktop/University/COURSE DOCS/THESIS/Internship/musow-pipeline/MJI/MJI_additions_for_LR.csv')
mji_additions_1['Title'] = mji_additions_1['Title'].str.lower().str.strip()
mji_additions_1['Description'] = mji_additions_1['Description'].str.lower().str.strip()
mji_additions_1['URL'] = mji_additions_1['URL'].str.lower().str.strip()
mji_additions_1 = mji_additions_1[~mji_additions_1['Title'].isin(df_musow['Title'])].dropna()
mji_additions_1 = mji_additions_1[~mji_additions_1['Title'].isin(mji_training_set['Title'])].dropna()
negative_df_adds = pd.concat([mji_training_set, mji_additions_1]).reset_index(drop=True)
negative_df_adds = negative_df_adds.drop_duplicates(['Title'], keep='last')
negative_df_adds['Target'] = '0'

In [7]:
#merge both sets into one, create two training sets one base, one extended w/ additional data, pickle for reuse
training_set = pd.concat([positive_df, negative_df])
training_set['Target'] = training_set['Target'].astype('int')
training_set = training_set.reset_index(drop=True)
training_set_adds = pd.concat([positive_df_adds, negative_df_adds])
training_set_adds['Target'] = training_set_adds['Target'].astype('int')
training_set_adds = training_set_adds.reset_index(drop=True)
training_set.to_pickle(path+'title_desc_url_trainingset.pkl')
training_set_adds.to_pickle(path+'title_desc_url_trainingset_extended.pkl')

In [8]:
#combine three feature column into one, create new DFs, pickle for reuse

training_set_xtra = training_set.copy()
training_set_adds_xtra = training_set_adds.copy()

#create combined columns for desc+headline and desc+headline_url
def tokenize_url(url:str):
    url=url.replace("https","")
    url=url.replace("http","")
    url=url.replace("www","")   
    url=re.sub("(\W|_)+"," ",url)
    return url

#create tokenized URL field
training_set_xtra['tokenized_url']=training_set_xtra['URL'].apply(lambda x:tokenize_url(x))
#description + tokenized url
training_set_xtra['text_desc_headline_url'] = training_set_xtra['Description'] + ' '+ training_set_xtra['Title']+" " + training_set_xtra['tokenized_url']
training_set_xtra.drop(['tokenized_url', 'Title', 'Description', 'URL'], inplace=True, axis=1)

#create tokenized URL field
training_set_adds_xtra['tokenized_url']=training_set_adds_xtra['URL'].apply(lambda x:tokenize_url(x))
#description + tokenized url
training_set_adds_xtra['text_desc_headline_url'] = training_set_adds_xtra['Description'] + ' '+ training_set_adds_xtra['Title']+" " + training_set_adds_xtra['tokenized_url']
training_set_adds_xtra.drop(['tokenized_url', 'Title', 'Description', 'URL'], inplace=True, axis=1)

training_set_xtra.to_pickle(path+'title_desc_url_trainingset_xtracols.pkl')
training_set_adds_xtra.to_pickle(path+'title_desc_url_trainingset_extended_xtracols.pkl')

In [9]:
#print some base stats to keep track of changes: training set size, avg length of description (main feature)

print('Size of base set:', len(training_set.index), '\nMedian length of description in base set is:', training_set['Description'].str.len().median(), '\nMean length of description in base set is:', training_set['Description'].str.len().mean(), '\nSize of extended set:', len(training_set_adds.index), '\nMedian length of description in extended set is:', training_set_adds['Description'].str.len().median(), '\nMean length of description in extended set is:', training_set_adds['Description'].str.len().mean())

#print some base stats to keep track of changes: avg length of combined desc+title+url (main feature)

print('\nMedian length of description in base set w/ xtra cols is:', training_set_xtra['text_desc_headline_url'].str.len().median(), '\nMean length of description in base set w/ xtra cols is:', training_set_xtra['text_desc_headline_url'].str.len().mean(), '\nMedian length of description in extended set w/ xtra cols is:', training_set_adds_xtra['text_desc_headline_url'].str.len().median(), '\nMean length of description in extended set w/ xtra cols is:', training_set_adds_xtra['text_desc_headline_url'].str.len().mean())

Size of base set: 626 
Median length of description in base set is: 291.0 
Mean length of description in base set is: 343.55271565495207 
Size of extended set: 781 
Median length of description in extended set is: 355.0 
Mean length of description in extended set is: 535.2394366197183

Median length of description in base set w/ xtra cols is: 360.5 
Mean length of description in base set w/ xtra cols is: 408.073482428115 
Median length of description in extended set w/ xtra cols is: 419.0 
Mean length of description in extended set w/ xtra cols is: 601.8143405889884


Part 2:
- Try different unsupervised encoding approaches for categorical variables including OrdinalEncoder, Label Encoding, Tf-Idf, BackwardDifferenceEncoder 
- Ordinal and Label encoding use title, desc, and url + combined title/desc and title/desc/url as features depending on source version of training set
- BackwardDifference uses title, desc, and url + separated title/desc and title/desc/url as features (in order to see if there's any diff btw the two different combination of strings approach)
- Tf-Idf uses only desc, title/desc, and title/desc/url as features 

In [10]:
#encode categorial variables w/ OrdinalEncoder, pickle
ord_enc = OrdinalEncoder()

#base set 
training_set_ordinal = training_set.copy()
training_set_ordinal["Title_encoded"] = ord_enc.fit_transform(training_set_ordinal[["Title"]])
training_set_ordinal["Desc_encoded"] = ord_enc.fit_transform(training_set_ordinal[["Description"]])
training_set_ordinal["URL_encoded"] = ord_enc.fit_transform(training_set_ordinal[["URL"]])
training_set_ordinal.to_pickle(path+'title_desc_url_trainingset_ordinal_encode.pkl')

#base set xtra columns
training_set_ordinal_xtra = training_set_xtra.copy()
training_set_ordinal_xtra["text_desc_headline_url_encoded"] = ord_enc.fit_transform(training_set_ordinal_xtra[["text_desc_headline_url"]])
training_set_ordinal_xtra.to_pickle(path+'title_desc_url_trainingset_xtracols_ordinal_encode.pkl')

#extended set 
training_set_ordinal_adds = training_set_adds.copy()
training_set_ordinal_adds["Title_encoded"] = ord_enc.fit_transform(training_set_ordinal_adds[["Title"]])
training_set_ordinal_adds["Desc_encoded"] = ord_enc.fit_transform(training_set_ordinal_adds[["Description"]])
training_set_ordinal_adds["URL_encoded"] = ord_enc.fit_transform(training_set_ordinal_adds[["URL"]])
training_set_ordinal_adds.to_pickle(path+'title_desc_url_trainingset_extended_ordinal_encode.pkl')

#extended set xtra columns 
training_set_ordinal_adds_xtra = training_set_adds_xtra.copy()
training_set_ordinal_adds_xtra["text_desc_headline_url_encoded"] = ord_enc.fit_transform(training_set_ordinal_adds_xtra[["text_desc_headline_url"]])
training_set_ordinal_adds_xtra.to_pickle(path+'title_desc_url_trainingset_extended_xtracols_ordinal_encode.pkl')

In [11]:
#encode cat variables with label encoding, pickle 

#base set 
training_set_label = training_set.copy()
training_set_label["Title"] = training_set_label["Title"].astype('category')
training_set_label["Description"] = training_set_label["Description"].astype('category')
training_set_label["URL"] = training_set_label["URL"].astype('category')
training_set_label["Title_cat"] = training_set_label["Title"].cat.codes
training_set_label["Desc_cat"] = training_set_label["Description"].cat.codes
training_set_label["URL_cat"] = training_set_label["URL"].cat.codes
training_set_label.to_pickle(path+'title_desc_url_trainingset_label_encode.pkl')

#base set xtra cols 
training_set_label_xtra = training_set_xtra.copy()
training_set_label_xtra["text_desc_headline_url"] = training_set_label_xtra["text_desc_headline_url"].astype('category')
training_set_label_xtra["text_desc_headline_url_cat"] = training_set_label_xtra["text_desc_headline_url"].cat.codes
training_set_label_xtra.to_pickle(path+'title_desc_url_trainingset_xtracols_label_encode.pkl')

#extended set 
training_set_label_adds = training_set_adds.copy()
training_set_label_adds["Title"] = training_set_label_adds["Title"].astype('category')
training_set_label_adds["Description"] = training_set_label_adds["Description"].astype('category')
training_set_label_adds["URL"] = training_set_label_adds["URL"].astype('category')
training_set_label_adds["Title_cat"] = training_set_label_adds["Title"].cat.codes
training_set_label_adds["Desc_cat"] = training_set_label_adds["Description"].cat.codes
training_set_label_adds["URL_cat"] = training_set_label_adds["URL"].cat.codes
training_set_label_adds.to_pickle(path+'title_desc_url_trainingset_extended_label_encode.pkl')

#extended set xtra cols 
training_set_label_adds_xtra = training_set_adds_xtra.copy()
training_set_label_adds_xtra["text_desc_headline_url"] = training_set_label_adds_xtra["text_desc_headline_url"].astype('category')
training_set_label_adds_xtra["text_desc_headline_url_cat"] = training_set_label_adds_xtra["text_desc_headline_url"].cat.codes
training_set_label_adds_xtra.to_pickle(path+'title_desc_url_trainingset_extended_xtracols_label_encode.pkl')

In [12]:
#encode w/ CE's BackwardDifferenceEncoder, pickle 

#base set 
training_set_ce = training_set.copy()
# Specify the columns to encode then fit and transform
encoder_base = ce.BackwardDifferenceEncoder(cols=['Title', 'Description', 'URL'])
training_set_ce = encoder_base.fit_transform(training_set_ce, verbose=1)
training_set_ce.drop(['intercept'], inplace=True, axis=1)
training_set_ce.to_pickle(path+'title_desc_url_trainingset_BD_encode.pkl')

#extended set 
training_set_adds_ce = training_set_adds.copy()
training_set_adds_ce = encoder_base.fit_transform(training_set_adds_ce, verbose=1)
training_set_adds_ce.drop(['intercept'], inplace=True, axis=1)
training_set_adds_ce.to_pickle(path+'title_desc_url_trainingset_extended_BD_encode.pkl')

#base set xtra cols, two types (urls and no urls)
training_set_xtra_ce = training_set_xtra.copy()
encoder_xtra_url = ce.BackwardDifferenceEncoder(cols=['text_desc_headline_url'])
training_set_xtra_ce_url = encoder_xtra_url.fit_transform(training_set_xtra_ce, verbose=1)
training_set_xtra_ce_url.drop(['intercept'], inplace=True, axis=1)
training_set_xtra_ce_url.to_pickle(path+'title_desc_url_trainingset_xtracols_BD_encode_url.pkl')

#extended set xtra cols 
training_set_adds_xtra_ce = training_set_adds_xtra.copy()
training_set_adds_xtra_ce_url = encoder_xtra_url.fit_transform(training_set_adds_xtra_ce, verbose=1)
training_set_adds_xtra_ce_url.drop(['intercept'], inplace=True, axis=1)
training_set_adds_xtra_ce_url.to_pickle(path+'title_desc_url_trainingset_extended_xtracols_BD_encode_url.pkl')


In [2]:
#ADD PICKLE READING CHUNK HERE TO NOT RERUN ALL ABOVE CODE
training_set = pd.read_pickle(path+'title_desc_url_trainingset.pkl')
training_set_adds = pd.read_pickle(path+'title_desc_url_trainingset_extended.pkl')
training_set_xtra = pd.read_pickle(path+'title_desc_url_trainingset_xtracols.pkl')
training_set_adds_xtra = pd.read_pickle(path+'title_desc_url_trainingset_extended_xtracols.pkl')
training_set_ordinal = pd.read_pickle(path+'title_desc_url_trainingset_ordinal_encode.pkl')
training_set_ordinal_xtra = pd.read_pickle(path+'title_desc_url_trainingset_xtracols_ordinal_encode.pkl')
training_set_ordinal_adds = pd.read_pickle(path+'title_desc_url_trainingset_extended_ordinal_encode.pkl')
training_set_ordinal_adds_xtra = pd.read_pickle(path+'title_desc_url_trainingset_extended_xtracols_ordinal_encode.pkl')
training_set_label = pd.read_pickle(path+'title_desc_url_trainingset_label_encode.pkl')
training_set_label_xtra = pd.read_pickle(path+'title_desc_url_trainingset_xtracols_label_encode.pkl')
training_set_label_adds = pd.read_pickle(path+'title_desc_url_trainingset_extended_label_encode.pkl')
training_set_label_adds_xtra = pd.read_pickle(path+'title_desc_url_trainingset_extended_xtracols_label_encode.pkl')
training_set_ce = pd.read_pickle(path+'title_desc_url_trainingset_BD_encode.pkl')
training_set_adds_ce = pd.read_pickle(path+'title_desc_url_trainingset_extended_BD_encode.pkl')
training_set_xtra_ce_url = pd.read_pickle(path+'title_desc_url_trainingset_xtracols_BD_encode_url.pkl')
training_set_adds_xtra_ce_url = pd.read_pickle(path+'title_desc_url_trainingset_extended_xtracols_BD_encode_url.pkl')

In [4]:
#encode w/ tf-idf

#create vectorizer 
tfidf_vectorizer=TfidfVectorizer(use_idf=True, max_df=0.95)

#base set description 
tfidf_vectorizer.fit_transform(training_set['Description'].values)
training_set_tfidf = tfidf_vectorizer.transform(training_set['Description'].values)

#extended set description 
tfidf_vectorizer.fit_transform(training_set_adds['Description'].values)
training_set_adds_tfidf = tfidf_vectorizer.transform(training_set_adds['Description'].values)

#base set xtra cols description+title+url
tfidf_vectorizer.fit_transform(training_set_xtra['text_desc_headline_url'].values)
training_set_xtra_url_tfidf = tfidf_vectorizer.transform(training_set_xtra['text_desc_headline_url'].values)

#base set xtra cols description+title+url
tfidf_vectorizer.fit_transform(training_set_adds_xtra['text_desc_headline_url'].values)
training_set_adds_xtra_url_tfidf = tfidf_vectorizer.transform(training_set_adds_xtra['text_desc_headline_url'].values)


In [6]:
#set features and targets for all encoding types 

#ordinal encoding 
feature_cols_ord = ['Title_encoded', 'Desc_encoded', 'URL_encoded']
feature_cols_ord_comb = ['text_desc_headline_url_encoded']

#base set
x_ord = training_set_ordinal[feature_cols_ord] # Features
y_ord = training_set_ordinal.Target # Target variable

#extended set 
x_ord_adds = training_set_ordinal_adds[feature_cols_ord] # Features
y_ord_adds = training_set_ordinal_adds.Target # Target variable

#base set xtra cols 
x_ord_xtra = training_set_ordinal_xtra[feature_cols_ord_comb] # Features
y_ord_xtra = training_set_ordinal_xtra.Target # Target variable

#extended set xtra cols 
x_ord_xtra_adds = training_set_ordinal_adds_xtra[feature_cols_ord_comb] # Features
y_ord_xtra_adds = training_set_ordinal_adds_xtra.Target # Target variable

#feature encoding 
feature_cols_label = ['Title_cat', 'Desc_cat', 'URL_cat']
feature_cols_label_comb = ['text_desc_headline_url_cat']

#base set
x_label = training_set_label[feature_cols_label] # Features
y_label = training_set_label.Target # Target variable

#extended set
x_label_adds = training_set_label_adds[feature_cols_label] # Features
y_label_adds = training_set_label_adds.Target # Target variable

#base set xtra cols 
x_label_xtra = training_set_label_xtra[feature_cols_label_comb] # Features
y_label_xtra = training_set_label_xtra.Target # Target variable

#extended set xtra cols 
x_label_xtra_adds = training_set_label_adds_xtra[feature_cols_label_comb] # Features
y_label_xtra_adds = training_set_label_adds_xtra.Target # Target variable

#BD encoding 

#base set 
x_backwards = training_set_ce.iloc[:,0:1825]
y_backwards = training_set_ce.Target

#extended set 
x_backwards_adds = training_set_adds_ce.iloc[:,0:2293]
y_backwards_adds = training_set_adds_ce.Target

#base xtra url 
x_backwards_xtra_url = training_set_xtra_ce_url.iloc[:,2:627]
y_backwards_xtra_url = training_set_xtra_ce_url.Target

#extended xtra url 
x_backwards_adds_xtra_url = training_set_adds_xtra_ce_url.iloc[:,2:782]
y_backwards_adds_xtra_url = training_set_adds_xtra_ce_url.Target

#tf-idf encoding 

#base set 
x_tfidf = training_set_tfidf
y_tfidf = training_set['Target'].values

#extended set 
x_adds_tfidf = training_set_adds_tfidf
y_adds_tfidf = training_set_adds['Target'].values

#base set xtra url 
x_tfidf_xtra_url = training_set_xtra_url_tfidf
y_tfidf_xtra_url = training_set_xtra['Target'].values

#extended set xtra url 
x_adds_tfidf_xtra_url = training_set_adds_xtra_url_tfidf
y_adds_tfidf_xtra_url = training_set_adds_xtra['Target'].values

Part 3: 
- run all variations of training set on two different types of Log Reg: cross eval and train/test 

In [7]:
#Log Reg cross eval 
def lr(x,y,title):  
    """ logistic regression"""
    model = LogisticRegression(solver='liblinear', C=10.0,random_state=44)
    y_pred = cross_val_predict(model, x, y, cv=10)
    acc = cross_val_score(model, x, y, cv=10, scoring='precision')    
    report = classification_report(y, y_pred)
    return print(f'{title}\n''MEAN PRECISION', np.mean(acc), 'report:', report, sep='\n')

In [8]:
#Ordinal Encoding results with crossval
print(lr(x_ord, y_ord, 'Base Set Ordinal:'), lr(x_ord_adds, y_ord_adds, 'Extended Set Ordinal:'), lr(x_ord_xtra, y_ord_xtra, 'Base Set Xtra Col Ordinal:'), lr(x_ord_xtra_adds, y_ord_xtra_adds, 'Extended Set Xtra Col Ordinal:')) 

Base Set Ordinal:
MEAN PRECISION
0.7930343559137847
report:
              precision    recall  f1-score   support

           0       0.12      0.02      0.03       128
           1       0.79      0.97      0.87       498

    accuracy                           0.78       626
   macro avg       0.46      0.49      0.45       626
weighted avg       0.66      0.78      0.70       626

Extended Set Ordinal:
MEAN PRECISION
0.7504776818976107
report:
              precision    recall  f1-score   support

           0       0.60      0.47      0.53       267
           1       0.75      0.83      0.79       514

    accuracy                           0.71       781
   macro avg       0.67      0.65      0.66       781
weighted avg       0.70      0.71      0.70       781



/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _wa

Base Set Xtra Col Ordinal:
MEAN PRECISION
0.7955453149001535
report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       128
           1       0.80      1.00      0.89       498

    accuracy                           0.80       626
   macro avg       0.40      0.50      0.44       626
weighted avg       0.63      0.80      0.70       626

Extended Set Xtra Col Ordinal:
MEAN PRECISION
0.6581304771178189
report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       267
           1       0.66      1.00      0.79       514

    accuracy                           0.66       781
   macro avg       0.33      0.50      0.40       781
weighted avg       0.43      0.66      0.52       781

None None None None


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _wa

In [9]:
#Label Encoding results with crossval
print(lr(x_label, y_label, 'Base Set Label:'), lr(x_label_adds, y_label_adds, 'Extended Set Label:'), lr(x_label_xtra, y_label_xtra, 'Base Set Xtra Col Label:'), lr(x_label_xtra_adds, y_label_xtra_adds, 'Extended Set Xtra Col Label:')) 

Base Set Label:
MEAN PRECISION
0.7930343559137847
report:
              precision    recall  f1-score   support

           0       0.12      0.02      0.03       128
           1       0.79      0.97      0.87       498

    accuracy                           0.78       626
   macro avg       0.46      0.49      0.45       626
weighted avg       0.66      0.78      0.70       626

Extended Set Label:
MEAN PRECISION
0.7504776818976107
report:
              precision    recall  f1-score   support

           0       0.60      0.47      0.53       267
           1       0.75      0.83      0.79       514

    accuracy                           0.71       781
   macro avg       0.67      0.65      0.66       781
weighted avg       0.70      0.71      0.70       781

Base Set Xtra Col Label:
MEAN PRECISION
0.7955453149001535
report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       128
           1       0.80      1.00      0.89       

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _wa

Extended Set Xtra Col Label:
MEAN PRECISION
0.6581304771178189
report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       267
           1       0.66      1.00      0.79       514

    accuracy                           0.66       781
   macro avg       0.33      0.50      0.40       781
weighted avg       0.43      0.66      0.52       781

None None None None


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _wa

In [11]:
#BD Encoding results with crossval
print(lr(x_backwards, y_backwards, 'Base Set BD:'), lr(x_backwards_adds, y_backwards_adds, 'Extended Set BD:'), lr(x_backwards_xtra_url, y_backwards_xtra_url, 'Base Set Xtra Col BD URL:'), lr(x_backwards_adds_xtra_url, y_backwards_adds_xtra_url, 'Extended Set Xtra Col BD URL:')) 

Base Set BD:
MEAN PRECISION
0.9883720930232558
report:
              precision    recall  f1-score   support

           0       0.71      0.96      0.82       128
           1       0.99      0.90      0.94       498

    accuracy                           0.91       626
   macro avg       0.85      0.93      0.88       626
weighted avg       0.93      0.91      0.92       626

Extended Set BD:
MEAN PRECISION
0.9800000000000001
report:
              precision    recall  f1-score   support

           0       0.90      0.95      0.93       267
           1       0.97      0.95      0.96       514

    accuracy                           0.95       781
   macro avg       0.94      0.95      0.94       781
weighted avg       0.95      0.95      0.95       781

Base Set Xtra Col BD URL:
MEAN PRECISION
0.9892857142857142
report:
              precision    recall  f1-score   support

           0       0.78      0.95      0.86       128
           1       0.99      0.93      0.96       498



In [ ]:
#Tf-Idf Encoding results with crossval
print(lr(x_tfidf, y_tfidf, 'Base Set TF:'), lr(x_adds_tfidf, y_adds_tfidf, 'Extended Set TF:'), lr(x_tfidf_xtra_url, y_tfidf_xtra_url, 'Base Set Xtra Col TF URL:'), lr(x_adds_tfidf_xtra_url, y_adds_tfidf_xtra_url, 'Extended Set Xtra Col TF URL:')) 

In [13]:
#LogReg train/test 

def lr_training(x,y,title):
    x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=0)
    logreg = LogisticRegression()
    # fit the model with data
    logreg.fit(x_train,y_train)
    y_pred=logreg.predict(x_test)
    cnf_matrix = confusion_matrix(y_test, y_pred)
    return print(f'{title}\n' "Accuracy:", accuracy_score(y_test, y_pred), "\nPrecision:", precision_score(y_test, y_pred), "\nRecall:", recall_score(y_test, y_pred)), cnf_matrix

In [14]:
#Ordinal encoding results w/ train/test 
print(lr_training(x_ord, y_ord, 'Base Set Ordinal:'), lr_training(x_ord_adds, y_ord_adds, 'Extended Set Ordinal:'), lr_training(x_ord_xtra, y_ord_xtra, 'Base Set Xtra Col Ordinal:'), lr_training(x_ord_xtra_adds, y_ord_xtra_adds, 'Extended Set Xtra Col Ordinal:')) 

Base Set Ordinal:
Accuracy: 0.7834394904458599 
Precision: 0.7973856209150327 
Recall: 0.976
Extended Set Ordinal:
Accuracy: 0.6938775510204082 
Precision: 0.7445255474452555 
Recall: 0.8031496062992126
Base Set Xtra Col Ordinal:
Accuracy: 0.7961783439490446 
Precision: 0.7961783439490446 
Recall: 1.0
Extended Set Xtra Col Ordinal:
Accuracy: 0.6479591836734694 
Precision: 0.6479591836734694 
Recall: 1.0
(None, array([[  1,  31],
       [  3, 122]])) (None, array([[ 34,  35],
       [ 25, 102]])) (None, array([[  0,  32],
       [  0, 125]])) (None, array([[  0,  69],
       [  0, 127]]))


In [15]:
#Label Encoding results w/ train/test
print(lr_training(x_label, y_label, 'Base Set Label:'), lr_training(x_label_adds, y_label_adds, 'Extended Set Label:'), lr_training(x_label_xtra, y_label_xtra, 'Base Set Xtra Col Label:'), lr_training(x_label_xtra_adds, y_label_xtra_adds, 'Extended Set Xtra Col Label:')) 

Base Set Label:
Accuracy: 0.7834394904458599 
Precision: 0.7973856209150327 
Recall: 0.976
Extended Set Label:
Accuracy: 0.6938775510204082 
Precision: 0.7445255474452555 
Recall: 0.8031496062992126
Base Set Xtra Col Label:
Accuracy: 0.7961783439490446 
Precision: 0.7961783439490446 
Recall: 1.0
Extended Set Xtra Col Label:
Accuracy: 0.6479591836734694 
Precision: 0.6479591836734694 
Recall: 1.0
(None, array([[  1,  31],
       [  3, 122]])) (None, array([[ 34,  35],
       [ 25, 102]])) (None, array([[  0,  32],
       [  0, 125]])) (None, array([[  0,  69],
       [  0, 127]]))


In [16]:
#BD Encoding results with crossval
print(lr_training(x_backwards, y_backwards, 'Base Set BD:'), lr_training(x_backwards_adds, y_backwards_adds, 'Extended Set BD:'), lr_training(x_backwards_xtra_url, y_backwards_xtra_url, 'Base Set Xtra Col BD URL:'), lr_training(x_backwards_adds_xtra_url, y_backwards_adds_xtra_url, 'Extended Set Xtra Col BD URL:')) 

Base Set BD:
Accuracy: 1.0 
Precision: 1.0 
Recall: 1.0
Extended Set BD:
Accuracy: 1.0 
Precision: 1.0 
Recall: 1.0
Base Set Xtra Col BD URL:
Accuracy: 1.0 
Precision: 1.0 
Recall: 1.0
Extended Set Xtra Col BD URL:
Accuracy: 1.0 
Precision: 1.0 
Recall: 1.0
(None, array([[ 32,   0],
       [  0, 125]])) (None, array([[ 69,   0],
       [  0, 127]])) (None, array([[ 32,   0],
       [  0, 125]])) (None, array([[ 69,   0],
       [  0, 127]]))


In [17]:
#Tf-Idf Encoding results with crossval
print(lr_training(x_tfidf, y_tfidf, 'Base Set TF:'), lr_training(x_adds_tfidf, y_adds_tfidf, 'Extended Set TF:'), lr_training(x_tfidf_xtra_url, y_tfidf_xtra_url, 'Base Set Xtra Col TF URL:'), lr_training(x_adds_tfidf_xtra_url, y_adds_tfidf_xtra_url, 'Extended Set Xtra Col TF URL:')) 

Base Set TF:
Accuracy: 0.8089171974522293 
Precision: 0.8064516129032258 
Recall: 1.0
Extended Set TF:
Accuracy: 0.7806122448979592 
Precision: 0.78 
Recall: 0.9212598425196851
Base Set Xtra Col TF URL:
Accuracy: 0.8089171974522293 
Precision: 0.8064516129032258 
Recall: 1.0
Extended Set Xtra Col TF URL:
Accuracy: 0.7602040816326531 
Precision: 0.7666666666666667 
Recall: 0.905511811023622
(None, array([[  2,  30],
       [  0, 125]])) (None, array([[ 36,  33],
       [ 10, 117]])) (None, array([[  2,  30],
       [  0, 125]])) (None, array([[ 34,  35],
       [ 12, 115]]))
